In [ ]:
import spacy
from spacy.lang.en import English

In [ ]:
# ein paar versuche

enable = ["parser", "ner", "senter"]
# um basierend auf den "aktiven" Komponenten auf die "inaktiven" zu schließen müssen
# müssen wir die pipeline einmal komplett laden
nlp = spacy.load("en_core_web_sm")

components = [component[0] for component in nlp.components]

exclude = [component for component in components if component not in enable]

# ermitteln welche Komponenten nicht geladen werden
# exclude = [proc for proc in nlp.pipe_names if proc not in enable]
# print(exclude)

config = {
    "name": "en_core_web_sm",
    "disable": [],
    "exclude": exclude,
    "config": {
        "nlp.batch_size": 256,
        "components": {
            "attribute_ruler": {"validate": False},
            "lemmatizer": {"mode": "rule", "model": None, "overwrite": False},
            "parser": {
                "moves": None,
                "min_action_freq": 12,
                "learn_tokens": False,
                "update_with_oracle_cut_size": 200,
            },
        },
    },
}

# tmp = config.copy()
# da wir die pipeline haben können wir aus der Config spezifisch nur die Modifikationen laden die
# für das gegebene Model zulässig sind
# poplist = [key for key in config["config"]["components"] if key not in nlp.pipe_names]
# for key in poplist:
#    tmp["config"]["components"].pop(key)

nlp = spacy.load(**config)
print(nlp.pipe_names)

text = "This is a sentence. This is yet another sentence."
doc = nlp(text)

# for token in doc:
#    print(token.text, token.lemma_, token.norm_)
tmp = []
for sent in doc.sents:
    print(sent.text)

# Test on text from tutorial

In [ ]:
def get_sample_text(path):
    data = []
    i = 0
    inpar = False
    with open(path, "r") as myfile:
        for line in myfile:
            if line.startswith("<"):
                if inpar is False:
                    inpar = True
                    data.append(["", "", ""])
                    data[i][0] += line.replace("\n", " ")
                elif inpar is True:
                    inpar = False
                    data[i][2] += line.replace("\n", " ")
                    i += 1
            elif inpar:
                data[i][1] += line.replace("\n", " ")
    return data


data = get_sample_text("../data/Original/plenary.vrt")

In [ ]:
print(len(data))

for elem in data:
    print(elem[:5])

In [ ]:
nlp = spacy.load("en_core_web_sm")

doc1 = nlp(data)

In [ ]:
for sent in doc1.sents:
    for token in sent:
        print(token.i, token.text, token.ent_type_)

In [ ]:
print(type(data))

# Sentencizer
https://spacy.io/usage/linguistic-features#sbd

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("This is a sentence. This is another sentence.")
assert doc.has_annotation("SENT_START")
for sent in doc.sents:
    print(sent.text)

In [ ]:
name = "../data/Original/iued_test_original.txt"
# name = "../data/Original/iued_test_original.vrt"
with open(name, "r") as myfile:
    data = myfile.read().replace("\n", "")

In [ ]:
print(data)

In [ ]:
doc = nlp(data)
assert doc.has_annotation("SENT_START")
for sent in doc.sents:
    print(sent.text)
    print("***")

This gives somewhat accurate results, with some errors after numbers. You can also use a trained model, however this will not work on uncommon texts.

In [ ]:
doc = nlp(data)
for sent in doc.sents:
    print(sent.text)
    print("***")

Also fails for the example here. Then there is the one based on a statistical model.

In [ ]:
nlp.enable_pipe("senter")
doc = nlp(data)
for sent in doc.sents:
    print(sent.text)
    print("***")

Directly use the sentencizer without the pipeline - this one looks at punctuation.

In [ ]:
nlp = English()  # just the language with no pipeline
nlp.add_pipe("sentencizer")
doc = nlp(data)
for sent in doc.sents:
    print(sent.text)
    print("***")

Seems to work correctly. What is the difference to the pipeline? In the DW scripts, the other components are disabled via the "exclude" command - should be faster as pipeline is not loaded at all.

In [ ]:
texts = [
    "Net income was $9.4 million compared to the prior year of $2.7 million.",
    "Revenue exceeded twelve billion dollars, with a loss of $1b.",
]

nlp = spacy.load("en_core_web_sm")
for doc in nlp.pipe(
    texts, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]
):
    # Do something with the doc here
    print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
nlp = spacy.load(
    "en_core_web_sm", exclude=["tagger", "ner", "attribute_ruler", "lemmatizer"]
)
for doc in nlp.pipe(texts):
    for sent in doc.sents:
        print(sent.text)
        print("***")

In [ ]:
lang = spacy.blank("en")
for key in lang.factories:
    print(key, English.factories[key])

In [ ]:
nlp = spacy.load("en_core_web_sm")
# print(nlp.components)
used = ["tok2vec", "ner"]
disable = []
for tupple in nlp.components:
    if tupple[0] not in used:
        disable.append(tupple[0])

print(disable)

In [ ]:
factory = "ner"

In [ ]:
if factory in nlp.factories:
    print(True)

# Tokenizer
https://spacy.io/usage/linguistic-features#tokenization  
We need to allow for special case rules. 
```
special_case = [{ORTH: "gim"}, {ORTH: "me"}]
nlp.tokenizer.add_special_case("gimme", special_case)
```

Also, there are custom tokenizer libraries that one may want to load. Probably we would want to keep it so that users can specify their custom tokenizers in addition to the standard one from spaCy.

In [ ]:
doc = nlp(data)
for token in doc:
    print(token.text)

# Lemmatizer
https://spacy.io/usage/linguistic-features#lemmatization

needs package spacy_lookups_data to run

In [ ]:
lemmatizer = nlp.add_pipe(
    "lemmatizer"
)  # need to be carefull which components are already in the pipeline or not. get_pipe() throws me an error when running this from the top
print(lemmatizer.mode)  # 'rule'
lemmatizer.initialize(lookups=None)
doc = nlp("I was reading the paper.")
print([token.lemma_ for token in doc])

In [ ]:
doc = nlp(data)
print([token.lemma_ for token in doc])

Should punctuation be excluded?

# POS tagger
https://spacy.io/usage/linguistic-features#pos-tagging

In [ ]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for token in doc:
    print(
        token.text,
        token.lemma_,
        token.pos_,
        token.tag_,
        token.dep_,
        token.shape_,
        token.is_alpha,
        token.is_stop,
    )

In [ ]:
doc = nlp(data)
for token in doc:
    print(token.text, token.pos_, token.tag_)

# Morphology
https://spacy.io/usage/linguistic-features#morphology

In [ ]:
print("Pipeline:", nlp.pipe_names)
doc = nlp("I was reading the paper.")
token = doc[0]  # 'I'
print(token.morph)  # 'Case=Nom|Number=Sing|Person=1|PronType=Prs'
print(token.morph.get("PronType"))  # ['Prs']

# Constituency

# Collocation

# Word vectors

# Dependency

# Named entities

Needs a spaCy pipeline that supports entity recognition. After running the pipeline the named entities can be accesed via Doc.ents.

In [ ]:
from collections import defaultdict

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
# in our case
doc = nlp(data)

for ent in doc.ents:
    print(ent.text, ent.start, ent.end, ent.label_)

In [ ]:
def named_entities_spacy(doc):
    """Fetch named entities from spaCy Doc object.

    [Args]:
            doc[class object]: The doc object of the data of interest after application of a spaCy pipeline.

    [Returns]:
            [dict]: Dictionary containing the identified named entities by name and type of label given by
                    spaCy associated with a list containing the indices to the start and end char for each
                    individual instance."""

    # define defaultdict to store the named entities
    named_entities = defaultdict(list)

    for ent in doc.ents:
        # add the entities label, start_char and end_char to the dictionary
        named_entities[
            "Text: {} |Label: {} |IOB: {}".format(ent.text, ent.label_, ent.label)
        ].append([ent.start_char, ent.end_char])

    return named_entities

In [ ]:
named_ent = named_entities_spacy(doc)

In [ ]:
print(named_ent)

In [ ]:
for idx in named_ent["Text: Audi |Label: ORG |IOB: 383"]:
    print(data[idx[0] - 20 : idx[1] + 20])

In [ ]:
for token in doc:
    print(token.ent_type_)

# Unique Tokens or smthg :)


Nicht das was named entities eigentlich will...
Für einzelne Tokens: [Matcher](https://spacy.io/usage/rule-based-matching) \
Für ganze Sätze: [Phrasematcher](https://spacy.io/usage/rule-based-matching#phrasematcher) \
Ich schätze für den Moment sind wir nur an einzelnen Tokens interessiert? Oder an allen einzigartigen Token im ganzen Text?

In [ ]:
from spacy.matcher import Matcher

# this works on the same example as above

nlp = spacy.load("en_core_web_sm")
# initialize the matcher, the vocab has to be the same as for the text
matcher = Matcher(nlp.vocab)

# idealy the user specifies what he wants to search and what attribute to assign?
terms = [[{"LOWER": "audi"}], [{"LOWER": "improvements"}], [{"LOWER": "parking"}]]

# also supports regular expressions:
terms += [[{"TEXT": {"REGEX": r"^[Ii](\.?|f)$"}}]]  # search for I, i, If, if

print("Query: {}".format(terms))

# add the terms to look for to the mathcer
matcher.add("Query", terms)

# load the data into doc
doc = nlp(data)
# run the matcher on the text in doc
matches = matcher(doc)

# get the indices (would correspond to corpus position from cwb?)
indices = [[start, end] for _, start, end in matches]

print(indices)

dict_out = defaultdict(
    list
)  # default dict initializes the value of a new key that is added with an empty list
# which we can then append to

# put the found indices to access the searched terms in a dictionary where they are available via said terms
# We have to go through all the found entities to confirm what term they correspond to...
# For large texts where we have many hits faster to split the query beforehand? Maybe parallel searching?
for index_pair in indices:
    dict_out["{}".format(doc[index_pair[0] : index_pair[1]])].append(index_pair)

# display the output
for key in dict_out:
    print("{} found at location {}.".format(key, dict_out[key]))

# for index in indices:
#    print('{} found at index location {}'.format(doc[index], index))

# print(matches)

# for match_id, start, end in matches:
#    string_id = nlp.vocab.strings[match_id] #Get string representation
#    span = doc[start:end] # matched span
#    print(match_id, string_id, start, end, span.text)

In [ ]:
# can also search for words of certain length or above/below certain lengths
matcher = Matcher(nlp.vocab)

pattern = [[{"LENGTH": {"==": 10}}]]  # , [{"LENGTH":{"<=":1}}], [{"LENGTH":{">=":12}}]]

matcher.add("Query", pattern)
matches = matcher(doc)

indices = [[start, end] for _, start, end in matches]

dict_out = defaultdict(list)

for index_pair in indices:
    dict_out["{}".format(doc[index_pair[0] : index_pair[1]])].append(index_pair)

for key in dict_out:
    print("{} found at location {}.".format(key, dict_out[key]))

In [ ]:
# search for token pattern

matcher = Matcher(nlp.vocab)

# seach for the different types of cars with a "wildcard token pattern" leaving the last token empty
pattern = [[{"ORTH": "Audi"}, {"ORTH": "A"}, {}]]

matcher.add("Query", pattern)

matches = matcher(doc)

indices = [[start, end] for _, start, end in matches]

for _, start, end in matches:
    indices.append([start, end])

dict_out = defaultdict(list)

for index_pair in indices:
    dict_out["{}".format(doc[index_pair[0] : index_pair[1]])].append(index_pair)

for key in dict_out:
    print("{} found at location {}.".format(key, dict_out[key]))

In [ ]:
# put it in a function


def search_text(query, nlp, doc):
    matcher = Matcher(nlp.vocab)

    matcher.add("Query", query)

    matches = matcher(doc)

    indices = [[start, end] for _, start, end in matches]

    dict_out = defaultdict(list)

    for index_pair in indices:
        dict_out["{}".format(doc[index_pair[0] : index_pair[1]])].append(index_pair)

    # for key in dict_out:
    #    print("{} found at location {}.".format(key, dict_out[key]))
    return dict_out

In [ ]:
query = [[{"ORTH": "Audi"}, {"ORTH": "A"}, {}]]

test1 = search_text(query, nlp, doc)

print(test1)

print("*" * 50)

# can just add different queries together
query += terms

test2 = search_text(query, nlp, doc)

print(test2)

In [ ]:
def get_unique(doc):
    """Get number of unique words in doc"""

    seen = set()
    for token in doc:
        if token.text not in seen:
            seen.add(token.text)
    return seen

In [ ]:
def search_token(query, nlp, doc):
    """search text for specific token and return all the found locations in dict."""

    matcher = Matcher(nlp.vocab)

    matcher.add("Query", query)

    matches = matcher(doc)

    indices = [[start, end] for _, start, end in matches]

    dict_out = defaultdict(list)

    dict_out["{}".format(doc[indices[0][0] : indices[0][1]])].append(indices)

    # for key in dict_out:
    #    print("{} found at location {}.".format(key, dict_out[key]))
    return dict_out

In [ ]:
def unique_tokens(nlp, doc):
    """Get locations for all umique words in doc into a dictionary, case sensitive"""

    unique_tokens = None

    # get the number of unique tokens in text, so we don't index twice
    unique = get_unique(doc)

    for token in unique:

        if unique_tokens:
            # if the dictionary is not empty:
            if token in unique_tokens:
                # if the token is already in the dictionary:
                pass
            elif token not in unique_tokens:
                # if the token is not already in there add it:
                unique_tokens.update(
                    search_token([[{"ORTH": "{}".format(token)}]], nlp, doc)
                )
        else:
            # if the dictionary hasn't been initialized do so with first token
            unique_tokens = search_token([[{"ORTH": "{}".format(token)}]], nlp, doc)

        if len(unique_tokens) == unique:
            # if we have passed each unique word already there is no need to continue
            break

    return unique_tokens

In [ ]:
unique_tok = unique_tokens(nlp, doc)

In [ ]:
for key in named_ent:
    print("{}: {}".format(key, named_ent[key]))

In [ ]:
print(len(named_ent))

In [ ]:
print(named_ent["the"])

In [ ]:
int = input("Could not find on system. Attempt to download? [Y/N]")

# Basically just old unused stuff from class module beyond this point, but you never know.

    #def grab_NER(self):

        # store the named entities and associated information about label
    #    self.named_entities = dict()
        # extract the information such as Label (as text and IOB code)
        # and position in text (by start/end char and start/end text ID)
        # the ID should correspond to the id in XML i think
        #for ent in self.doc.ents:
            # if entity is one token we just append it
            #if ent.end - ent.start == 1:
                #self.named_entities["{}".format(ent.start)] = {
                    #"Label_text": ent.label_,
                    #"IOB": ent.label,
                    #"Chars": [ent.start_char, ent.end_char],
                    #"Token Ids": [ent.start, ent.end],
                    #"Text": ent.text,
                #}
            # if entity is more than one token we also add the Ids of
            # all contained tokens to the dict and assign them the same labels etc.
            #else:
                #for i in range(ent.start, ent.end):
                    #self.named_entities["{}".format(i)] = {
                        #"Label_text": ent.label_,
                        #"IOB": ent.label,
                        #"Chars": [ent.start_char, ent.end_char],
                        #"Token Ids": [ent.start, ent.end],
                        #"Text": ent.text,
                    #}

        #return self.named_entities

'''

def preprocess(spec_pipe=None):
    """Load model"""

    # Add possibility for user to provide their own spacy model or
    # use specific one?
    if spec_pipe:
        nlp = sp.load(spec_pipe)

    else:
        # just this one model for now, what models are needed? There
        # are a lot of different ones in the old preprocessing
        print("Loading model en_core_web_sm")
        nlp = sp.load("en_core_web_sm")

    return nlp


def pipeline_with_dict():

    # Information about pipeline and the components:
    # https://spacy.io/usage/processing-pipelines

    config = {
        "name":"en_core_web_sm", #name of model
        "disable":[], # components to be disabled, given as list, loaded but not used
        "exclude":["tagger","parser","attribute_ruler","lemmatizer"], # components to be excluded, given as list, wont be loaded
        "config":{}
    }# put the instructions for sp.laod() into dict

    nlp = sp.load(**config)
    return nlp


def blank_with_dict():
    # load a default language and then add components from a pretrained pipeline for
    # that language
    config = {
        "lang":"en",
        "add_to":["tok2vec", "tagger","parser","ner","attribute_ruler","lemmatizer"],
        "source":"en_core_web_sm"
    }# config equivalent to sp.load("en_core_web_sm")


    configNER = {
        "lang":"en",
        "add_to":["ner"],
        "source":"en_core_web_sm"
    }# load just NER from en_core_web_sm

    nlp = sp.blank(configNER["lang"])
    for factory in configNER["add_to"]:
        nlp.add_pipe(factory, source=sp.load(configNER["source"]))

    return nlp

def apply_pipe(data, nlp):
    """Apply given pipeline to data"""
    # for multiprocessing or if we have multiple documents we can consider
    # something like:
    # docs = [doc for doc in nlp.pipe(data, n_process=cores)]
    # this should stream the individual documents and create Docs in order while
    # working in parallel. Maybe break down large corpus into peaces for this?
    return nlp(data)


# I guess we want to eiter load a pipeline or build one and apply it
# so we can assume that doc exists and if NER is called, that
# the pipeline included 'ner'
def NER(doc):
    """Get the NER tokens from Doc"""

    # store the named entities and associated information about label
    named_entities = defaultdict(list)
    # extract the information such as Label (as text and IOB code)
    # and position in text (by start/end char and start/end text ID)
    # the ID should correspond to the id in XML i think
    for ent in doc.ents:
        named_entities["Start Id: {}".format(ent.start)].append(
            {
                "Label_text": ent.label_,
                "IOB": ent.label,
                "Chars": [ent.start_char, ent.end_char],
                "Token Ids": [ent.start, ent.end],
            }
        )

    return named_entities


if __name__ == "__main__":
    with open("../../data/Original/iued_test_original.txt", "r") as file:
        data = file.read().replace("\n", "")

    # use the regular model
    doc = apply_pipe(data, preprocess())
    # for token in doc:
    #   print(token.text)
    named_ents = NER(doc)
    for key in named_ents:
        print("{} {}".format(key, named_ents[key]))

    # also use regular model but exclude components
    doc2 = apply_pipe(data, pipeline_with_dict())
    print('*'*20)
    # get NER
    named_ents2 = NER(doc2)
    # check that NER still works
    for key in named_ents:
        print("{} {}".format(key, named_ents2[key]))
        if key in named_ents2:
            try:
                assert named_ents2[key] == named_ents[key]
            except AssertionError:
                print(named_ents[key], named_ents2[key])
        else:
            print("!Untracked token in ents2: {}".format(named_ents[key]))

    # "rebuilt" en_core_web_sm manually from basic sp.lang.en.English
    doc3 = apply_pipe(data, blank_with_dict())
    print('*'*20)
    named_ents3 = NER(doc3)
    # check that NER still works
    for key in named_ents3:
        print("{} {}".format(key, named_ents3[key]))
        if key in named_ents3:
            try:
                assert named_ents3[key] == named_ents[key]
            except AssertionError:
                print(named_ents[key], named_ents3[key])
        else:
            print("!Untracked token in ents3: {}".format(named_ents[key]))
    # -> NER still works fine, but labels one entity differently for some reason

'''


  #for sent in doc.sents:
    #    print("<\s>")
    #    for token in sent:
    #        print(token.i, token.text, token.ent_type_)
    #    print("<\s>")
    
    # either load pipe with config from above or load a complete pretrained pipe
    # this will load all components
    #results = spaCy_pipe(config).apply_to(data).grab_results()
    # , pretrained="en_core_web_sm")

    #for key, value in results["ner"].items():
    #    print(
    #        "ID {} : [IOB]= {} [Text]={}, [Label]={}".format(
    #            key, value["IOB"], value["Text"], value["Label_text"]
    #        )
    #    )

    #print("*" * 50)

    #different_config = {
    #        "config":{"lang": "en", "add_to": ["ner"], "source": "en_core_web_md"},
    #        "jobs": ["ner"],
    #        "pretrained": False
    #    }

    #results1 = spaCy_pipe(different_config).apply_to(data).grab_results()

    # print(results1)

    #for key, value in results1["ner"].items():
    #    print(
    #        "ID {} : [IOB]= {} [Text]={}, [Label]={}".format(
    #            key, value["IOB"], value["Text"], value["Label_text"]
    #        )
    #    )

        #def grab_results(self):

        # grab the results after running the pipeline, currently only for NER
    #    results = dict()
    #    if "ner" in self.jobs:
    #        results["ner"] = self.grab_NER()

    #    return results